In [1]:
#%%
import datetime
import pandas as pd
import numpy as np
import psycopg2
from utils.portfolio import *
from utils.base import *
from utils.report import *
from utils.config import *
from utils.instance import *
import os
import warnings
warnings.filterwarnings('ignore')



In [8]:
options, market = fetch_real_time_last_snapshot()
options = extract_call_options_features(options)
for i in range(len(options)):
    try:
      options['interest_rate_ask_price_1'][i]>1
    except:
        options['interest_rate_ask_price_1'][i] = np.nan

In [9]:
options.head(10)

,id_option,isin_option,symbol_option,name,open_price,close_price,last_trade_option,count,volume,value,...,min_allowed_price_base,in_the_money_percentage,ask_price_1_w_o_c,bid_price_1_w_o_c,leverage_ask_price_1,leverage_bid_price_1,leverage_last_trade,interest_rate_last_trade,interest_rate_ask_price_1,interest_rate_bid_price_1
0,12833810923774304,IRO9AGAH0161,ضترو4004,اختيارخ ص آگاه-10000-02/04/14,3770.0,4276.0,4300.0,4,165.0,705500000.0,...,NaN,NaN,4300.0,3500.0,NaN,NaN,NaN,NaN,NaN,NaN
1,54179059222245165,IRO9AGAH0181,ضترو4006,اختيارخ ص آگاه-12000-02/04/14,2300.0,2525.0,2850.0,17,342.0,863668000.0,...,NaN,NaN,2850.0,2551.0,NaN,NaN,NaN,NaN,NaN,NaN
2,9181823518983931,IRO9AGAH0191,ضترو4007,اختيارخ ص آگاه-13000-02/04/14,1799.0,1950.0,1999.0,25,291.0,567405000.0,...,NaN,NaN,2060.0,1851.0,NaN,NaN,NaN,NaN,NaN,NaN
3,63421907578271281,IRO9AGAH0201,ضترو4008,اختيارخ ص آگاه-14000-02/04/14,1700.0,1387.0,1385.0,21,1320.0,1830429000.0,...,NaN,NaN,1500.0,1371.0,NaN,NaN,NaN,NaN,NaN,NaN
4,54936847147016069,IRO9AGAH0211,ضترو4009,اختيارخ ص آگاه-15000-02/04/14,750.0,750.0,750.0,1,9.0,6750000.0,...,NaN,NaN,1400.0,760.0,NaN,NaN,NaN,NaN,NaN,NaN
5,6656725773350069,IRO9AGAH0221,ضترو4010,اختيارخ ص آگاه-16000-02/04/14,600.0,604.0,620.0,21,567.0,342347000.0,...,NaN,NaN,900.0,590.0,NaN,NaN,NaN,NaN,NaN,NaN
6,29567585335146943,IRO9AHRM6291,ضهرم7003,اختيارخ اهرم-8000-1402/07/23,19000.0,19644.0,20000.0,11,51.0,1001867000.0,...,22450.0,0.708349,20989.0,20300.0,1.308222,1.349841,1.371500,0.175331,0.59428,0.2944
7,51752770340789810,IRO9AHRM6301,ضهرم7004,اختيارخ اهرم-9000-1402/07/23,18000.0,18665.0,19600.0,16,181.0,3378397000.0,...,22450.0,0.671892,21000.0,19600.0,1.307537,1.398050,1.399490,0.355772,1.070041,0.363444
8,36915839123127808,IRO9AHRM6311,ضهرم7005,اختيارخ اهرم-10000-1402/07/23,16500.0,16500.0,16500.0,1,10.0,165000000.0,...,22450.0,0.635436,19500.0,18500.0,1.408117,1.481177,1.662424,-0.176636,0.651001,0.286609
9,42868252948624691,IRO9AHRM6321,ضهرم7006,اختيارخ اهرم-11000-1402/07/23,16000.0,16750.0,17500.0,2,2.0,33500000.0,...,22450.0,0.598979,18999.0,17700.0,1.445249,1.548123,1.567429,0.250649,0.779369,0.312891


In [4]:
options.columns


Index(['id_option', 'isin_option', 'symbol_option', 'name', 'open_price',
       'close_price', 'last_trade_option', 'count', 'volume', 'value',
       'low_price', 'high_price', 'yesterday_price', 'table_id', 'group_id',
       'max_allowed_price', 'min_allowed_price_option', 'last_ret',
       'close_ret', 'number_of_shares', 'market_cap', 'bid_price_5',
       'bid_vol_5', 'ask_price_5', 'ask_vol_5', 'bid_price_4', 'bid_vol_4',
       'ask_price_4', 'ask_vol_4', 'bid_price_2', 'bid_vol_2', 'ask_price_2',
       'ask_vol_2', 'bid_price_1', 'bid_vol_1', 'ask_price_1', 'ask_vol_1',
       'bid_price_3', 'bid_vol_3', 'ask_price_3', 'ask_vol_3', 'ask_value_1',
       'ask_value_2', 'ask_value_3', 'ask_value_4', 'ask_value_5',
       'bid_value_1', 'bid_value_2', 'bid_value_3', 'bid_value_4',
       'bid_value_5', 'time', 'date', 'jalali_date', 'base_symbol',
       'jalali_due_date', 'days_until_due', 'miladi_due_date',
       'due_date_price', 'last_trade_base', 'symbol_base', 'id_base'

In [5]:
options['spread_pct'] = 100* (options['ask_price_1'] / options['bid_price_1']  - 1)

In [6]:
options[(options['value']>200e7 )& (options['spread_pct'] > 10) & \
        (options['last_trade_option'] /options['ask_price_1'] > 0.95 )&\
        (options['interest_rate_ask_price_1'] > 0.7)]

,id_option,isin_option,symbol_option,name,open_price,close_price,last_trade_option,count,volume,value,...,in_the_money_percentage,ask_price_1_w_o_c,bid_price_1_w_o_c,leverage_ask_price_1,leverage_bid_price_1,leverage_last_trade,interest_rate_last_trade,interest_rate_ask_price_1,interest_rate_bid_price_1,spread_pct
21,42780287876795888,IRO9AHRM6591,ضهرم1228,اختيارخ اهرم-28000-1402/12/23,9999.0,10012.0,11596.0,28,341.0,3414083000.0,...,-0.02078,11597.0,10101.0,2.367706,2.712779,2.365471,0.927743,0.92621,0.738738,14.574149
221,2218242214087037,IRO9TAMN0081,ضستا2016,اختيارخ شستا-1300-1402/02/13,187.0,242.0,264.0,192,55807.0,13505483000.0,...,0.149771,278.0,246.0,5.505671,6.209052,5.791667,144.593132,1063.298395,10.453847,12.775573
358,9620871538213717,IRO9PTEH6641,ضترا3023,اختيارخ شتران-3750-1402/03/28,2300.0,2319.0,2500.0,26,1024.0,2374411000.0,...,0.377076,2550.0,1910.0,2.363218,3.148589,2.408000,0.618188,0.79381,-0.500134,33.233110
373,22615772467293315,IRO9FKHZ6421,ضخوز2037,اختيارخ فخوز-4000-1402/02/10,720.0,765.0,799.0,319,31294.0,23924511000.0,...,0.173212,799.0,720.0,6.061312,6.712531,6.055069,33.519109,36.183918,38023.938666,10.743855


In [7]:
pd.options.display.max_rows = 999
options.loc[21]

id_option                                42780287876795888
isin_option                                   IRO9AHRM6591
symbol_option                                     ضهرم1228
name                         اختيارخ اهرم-28000-1402/12/23
open_price                                          9999.0
close_price                                        10012.0
last_trade_option                                  11596.0
count                                                   28
volume                                               341.0
value                                         3414083000.0
low_price                                           9667.0
high_price                                         11596.0
yesterday_price                                       9415
table_id                                                 3
group_id                                                68
max_allowed_price                                 200000.0
min_allowed_price_option                               1